In [5]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
!pip install box2d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 41.9 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import gym
import numpy as np
import torch
import os
import multiprocessing
import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecNormalize

In [9]:
def evaluate_model(model, env, n_eval_episodes=10):
    """Evaluate the model on a given environment and return mean and std rewards."""
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes)
    return mean_reward, std_reward

# Function Def

In [10]:
def train_lunarlander(env, bs, lr):
    # Create environment and model for LunarLander

    env_lunar = make_vec_env("LunarLander-v3", n_envs=env)
    env_lunar = VecNormalize(env_lunar, norm_obs=True, norm_reward=True)
    log = "/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv"+str(env)+"B"+str(bs)+"LR"+str(lr)+"/"
    dqn_model = DQN("MlpPolicy", env_lunar, verbose=1, tensorboard_log=log,
                    learning_rate = lr, batch_size = bs, gamma= 0.99,
                    exploration_initial_eps = 1.0, exploration_final_eps= 0.01,
                    exploration_fraction = 1, train_freq = 10, target_update_interval = 1000, learning_starts = 50)

    eval_freq = 10000  # Evaluate every 10,000 steps
    n_eval_episodes = 10  # Number of episodes per evaluation
    total_timesteps = 200000 # Define the total number of timesteps

    for step in range(1, total_timesteps + 1, eval_freq):
        # Train the model
        dqn_model.learn(total_timesteps=eval_freq, reset_num_timesteps=False)

        # Evaluate the model on LunarLander
        mean_reward, std_reward = evaluate_model(dqn_model, env_lunar, n_eval_episodes)

        # Print evaluation results for LunarLander
        print(f"LunarLander - Step: {step} | Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")

        #Stop the loop once reached target timesteps
        if step == total_timesteps:
            break

    # Save model and normalization
    dqn_model.save(log)
    env_lunar.save(log+".pkl")  # This ensures it gets saved!

    print("Model and VecNormalize stats saved!")

# 1. envs=1, batch size= 64, lr= 1e-4

In [11]:
train_lunarlander(1, 64, 1e-4)

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0001/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 96       |
|    ep_rew_mean      | -177     |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 940      |
|    time_elapsed     | 0        |
|    total_timesteps  | 384      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0702   |
|    n_updates        | 33       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 89.5     |
|    ep_rew_mean      | -171     |
|    exploration_rate | 0.929    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1207     |
|    time_elapsed     | 0        |
|    total_timesteps  | 716      |
| train/              |    

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0001' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0001_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 2. envs=1, batch size= 64, lr= 3e-4

In [12]:
train_lunarlander(1, 64, 3e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0003/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 86.8     |
|    ep_rew_mean      | -134     |
|    exploration_rate | 0.966    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1859     |
|    time_elapsed     | 0        |
|    total_timesteps  | 347      |
| train/              |          |
|    learning_rate    | 0.0003   |
|    loss             | 0.0105   |
|    n_updates        | 29       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 99.8     |
|    ep_rew_mean      | -169     |
|    exploration_rate | 0.921    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1844     |
|    time_elapsed     | 0        |
|    total_timesteps  | 798      |
| train/              |    

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0003' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B64LR0.0003_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 3. envs=1, batch size= 128, lr= 1e-4

In [13]:
train_lunarlander(1, 128, 1e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0001/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 66.5     |
|    ep_rew_mean      | -78.8    |
|    exploration_rate | 0.974    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1767     |
|    time_elapsed     | 0        |
|    total_timesteps  | 266      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.188    |
|    n_updates        | 21       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 76.5     |
|    ep_rew_mean      | -144     |
|    exploration_rate | 0.939    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1693     |
|    time_elapsed     | 0        |
|    total_timesteps  | 612      |
| train/              |   

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0001' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0001_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 4. envs=1, batch size= 128, lr=3e-4

In [14]:
train_lunarlander(1, 128, 3e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0003/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 82.5     |
|    ep_rew_mean      | -202     |
|    exploration_rate | 0.967    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1850     |
|    time_elapsed     | 0        |
|    total_timesteps  | 330      |
| train/              |          |
|    learning_rate    | 0.0003   |
|    loss             | 0.00413  |
|    n_updates        | 27       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 86.4     |
|    ep_rew_mean      | -173     |
|    exploration_rate | 0.932    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1806     |
|    time_elapsed     | 0        |
|    total_timesteps  | 691      |
| train/              |   

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0003' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv1B128LR0.0003_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


#  5. envs=4, batch size= 64, lr=1e-4

In [15]:
train_lunarlander(4, 64, 1e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0001/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 98.2     |
|    ep_rew_mean      | -164     |
|    exploration_rate | 0.95     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3510     |
|    time_elapsed     | 0        |
|    total_timesteps  | 500      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00753  |
|    n_updates        | 11       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 104      |
|    ep_rew_mean      | -174     |
|    exploration_rate | 0.907    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3679     |
|    time_elapsed     | 0        |
|    total_timesteps  | 936      |
| train/              |    

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0001' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0001_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 6. envs=4, batch size= 64, lr=3e-4

In [16]:
train_lunarlander(4, 64, 3e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0003/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 106      |
|    ep_rew_mean      | -216     |
|    exploration_rate | 0.95     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3789     |
|    time_elapsed     | 0        |
|    total_timesteps  | 508      |
| train/              |          |
|    learning_rate    | 0.0003   |
|    loss             | 0.0143   |
|    n_updates        | 11       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 98.1     |
|    ep_rew_mean      | -176     |
|    exploration_rate | 0.912    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3644     |
|    time_elapsed     | 0        |
|    total_timesteps  | 892      |
| train/              |    

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0003' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B64LR0.0003_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 7. envs=4, batch size= 128, lr=1e-4

In [17]:
train_lunarlander(4, 128, 1e-4)

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0001/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 102      |
|    ep_rew_mean      | -175     |
|    exploration_rate | 0.95     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3053     |
|    time_elapsed     | 0        |
|    total_timesteps  | 508      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0412   |
|    n_updates        | 11       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 85.9     |
|    ep_rew_mean      | -166     |
|    exploration_rate | 0.922    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3198     |
|    time_elapsed     | 0        |
|    total_timesteps  | 788      |
| train/              |   

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0001' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0001_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


# 8. envs=4, batch size= 128, lr=3e-4

In [11]:
train_lunarlander(4, 128, 3e-4)

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/

Using cpu device
Logging to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0003/DQN_0
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 91.2     |
|    ep_rew_mean      | -128     |
|    exploration_rate | 0.956    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 506      |
|    time_elapsed     | 0        |
|    total_timesteps  | 444      |
| train/              |          |
|    learning_rate    | 0.0003   |
|    loss             | 0.00936  |
|    n_updates        | 10       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 103      |
|    ep_rew_mean      | -205     |
|    exploration_rate | 0.908    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 712      |
|    time_elapsed     | 1        |
|    total_timesteps  | 928      |
| train/              |   

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/save_util.py:281: UserWarning: Path '/content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0003' is a folder. Will save instead to /content/drive/MyDrive/RL_models/dqn_lunarlanderEnv4B128LR0.0003_2
  warnings.warn(f"Path '{path}' is a folder. Will save instead to {path}_2")


Model and VecNormalize stats saved!
